# Peter results

In [ ]:
import pymongo as pm
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import re

## Connect to db

In [ ]:
client = pm.MongoClient()   #We are using local host
db = client["experiment"]

##  Analysis

In [ ]:
a = db["sessions"].count_documents({})
a

### Finding the correct awnser to the problem

In [ ]:

# full_name = pd.DataFrame()
# single_letter = pd.DataFrame()
# abbreviation = pd.DataFrame()
# semantic = pd.DataFrame()
# syntatic = pd.DataFrame()
full_name = []
single_letter = []
abbreviation = []
semantic = []
syntatic = []
abbreviation_regex = "abbrev.html$"
full_name_regex = "normal.html$"
single_letter_regex = "single.html$"
semantic_regex = "..\.semantic.."
syntatic_regex = "..\.syntactic.."


sessions = db["sessions"]
for session in sessions.find({"done": True}):
    for trial in session["trials"]:
        # print(trial)
        # print((trial["file"]))
        if re.search(pattern=full_name_regex, string=trial["file"]):
            full_name.append(trial)
        elif re.search(pattern=abbreviation_regex, string=trial["file"]):
            abbreviation.append(trial)
        elif re.search(pattern=single_letter_regex, string=trial["file"]):
            single_letter.append(trial)
        if re.search(pattern=semantic_regex, string=trial["file"]):
            semantic.append(trial)
        elif re.search(pattern=syntatic_regex, string=trial["file"]):
            syntatic.append(trial)

In [ ]:
def success_rate(bucket: list) -> [list, list]:
    success = 0
    fail = 0
    for drop in bucket:
        if drop["state"] == "success":
            success += 1
        elif drop["state"] == "failure":
            fail += 1
    return [[success, fail], ["success", "failure"]]

In [ ]:
data, keys = success_rate(single_letter)

# define Seaborn color palette to use
palette_color = sns.color_palette('bright')

# plotting data on chart
plt.pie(data, labels=keys, colors=palette_color, autopct='%.0f%%')

# displaying chart
plt.show()